In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tensornetworks_pytorch.TNModels import PosMPS, Born
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

print(torch.__version__)

1.8.0.dev20201206


Use some data

In [2]:
import pickle
for dataset in [#'biofam',
    'flare','lymphography','spect','tumor','votes']:
    with open('datasets/'+dataset, 'rb') as f:
            a=pickle.load(f)
    X=a[0].astype(int)
    print(dataset)
    print("\tdata shape:", X.shape)
    print(f"\trange of X values: {X.min()} -- {X.max()}")

def load_dataset(dataset):
    with open('datasets/'+dataset, 'rb') as f:
            a=pickle.load(f)
    X=a[0]
    X=X.astype(int)

    print("\tdata shape:", X.shape)
    print(f"\trange of X values: {X.min()} -- {X.max()} ==> d={X.max()+1}")
    d = X.max()+1
    return X, d

flare
	data shape: (1065, 13)
	range of X values: 0 -- 7
lymphography
	data shape: (148, 19)
	range of X values: 0 -- 7
spect
	data shape: (187, 23)
	range of X values: 0 -- 1
tumor
	data shape: (339, 17)
	range of X values: 0 -- 3
votes
	data shape: (435, 17)
	range of X values: 0 -- 2


In [3]:
dataset = 'spect'
print("dataset:", dataset)
X,d = load_dataset(dataset)

D = 8
mps       = PosMPS(X, d, D, homogeneous=False)
mps_hom   = PosMPS(X, d, D, homogeneous=True)

rBorn     = Born(X, d, D, dtype=torch.float, homogeneous=False, log_stability=False) 
rBorn_hom = Born(X, d, D, dtype=torch.float, homogeneous=True, log_stability=False) 

cBorn     = Born(X, d, D, dtype=torch.cfloat, homogeneous=False, log_stability=False)
cBorn_hom = Born(X, d, D, dtype=torch.cfloat, homogeneous=True, log_stability=False)

rBorn_s   = Born(X, d, D, dtype=torch.float, homogeneous=False, log_stability=True) 
rBorn_s_hom = Born(X, d, D, dtype=torch.float, homogeneous=True, log_stability=True) 

cBorn_s   = Born(X, d, D, dtype=torch.cfloat, homogeneous=False, log_stability=True)
cBorn_s_hom = Born(X, d, D, dtype=torch.cfloat, homogeneous=True, log_stability=True)

models     = (rBorn, cBorn, rBorn_s, cBorn_s, mps)
models_hom = (rBorn_hom, cBorn_hom, rBorn_s_hom, cBorn_s_hom, mps_hom)
print("Initializing models:")
for model in (*models, *models_hom):
    print(f"\t{model.core.shape} model type: {model.name}")
    for p in model.parameters():
                if torch.isnan(p).any():
                    print("Model weights contain a NaN value!")

dataset: spect
	data shape: (187, 23)
	range of X values: 0 -- 1 ==> d=2
Initializing models:
	torch.Size([23, 2, 8, 8]) model type: Born (torch.float32), Non-homogeneous
	torch.Size([23, 2, 8, 8]) model type: Born (torch.complex64), Non-homogeneous
	torch.Size([23, 2, 8, 8]) model type: Born (torch.float32), Non-homogeneous + log_stability
	torch.Size([23, 2, 8, 8]) model type: Born (torch.complex64), Non-homogeneous + log_stability
	torch.Size([23, 2, 8, 8]) model type: Positive MPS, Non-homogeneous
	torch.Size([2, 8, 8]) model type: Born (torch.float32), Homogeneous
	torch.Size([2, 8, 8]) model type: Born (torch.complex64), Homogeneous
	torch.Size([2, 8, 8]) model type: Born (torch.float32), Homogeneous + log_stability
	torch.Size([2, 8, 8]) model type: Born (torch.complex64), Homogeneous + log_stability
	torch.Size([2, 8, 8]) model type: Positive MPS, Homogeneous


In [4]:
def register_clip_hook(model, clip_val, verbose=False):
    """"Set up model parameters to clip gradient at `clip_val`"""
    for p in model.parameters():
        if p.dtype==torch.cfloat:
            p.register_hook(lambda grad: torch.complex(
                grad.real.clamp(-clip_val, clip_val),grad.imag.clamp(-clip_val, clip_val)))
            if verbose:
                p.register_hook(lambda grad: print('real grad range',grad.real.min(),grad.real.max()))
                p.register_hook(lambda grad: print('imag grad range',grad.imag.min(),grad.imag.max()))
        else:
            p.register_hook(lambda grad: torch.clamp(grad, -clip_val, clip_val))
            if verbose:
                p.register_hook(lambda grad: print('grad range',grad.min(),grad.max()))

In [5]:
def train(self, dataset, batchsize, max_epochs, plot=True, **optim_kwargs):
    trainloader = DataLoader(dataset, batch_size=batchsize, shuffle=True)
    #register_clip_hook(self, clip_val=1, verbose=True)
    optimizer = torch.optim.SGD(self.parameters(), **optim_kwargs)
    early_stopping_threshold = 0.0001 # min difference in epoch loss
    loss_values = [] # store by-epoch avg loss values
    print('╭───────────────────────────')
    print(f"│Training {self.name}.")
    av_batch_loss_running = -1000
    with tqdm(range(max_epochs), leave=True) as tepochs:
        for epoch in tepochs:
    #         print("epoch", epoch)
            batch_loss = []
            with tqdm(trainloader, unit="batch", leave=False, desc=f"epoch {epoch}") as tepoch:
                for batch in tepoch:
                    for p in self.parameters():
                        if torch.isnan(p).any():
                            print(" loss values:", *(f"{x:.3f}" for x in loss_values))
                            print("└────Stopped. Model weights contain a NaN value!")
                            if plot:
                                plt.plot(loss_values)
                                plt.show()
                            return loss_values
                    self.zero_grad()
                    neglogprob = 0
                    for i,x in enumerate(batch):
                        out = self(x)
                        neglogprob -= out
                    loss = neglogprob / len(batch)
                    for p in self.parameters():
                        if torch.isnan(p).any():
                            print(" loss values:", *(f"{x:.3f}" for x in loss_values))
                            print("└────Stopped. Model weights contain a NaN value after forward pass!")
                            if plot:
                                plt.plot(loss_values)
                                plt.show()
                            return loss_values
                    loss.backward()
                    optimizer.step()
                    tepoch.set_postfix(loss=loss.item())
                    with torch.no_grad():
                        batch_loss.append(loss.item())
    #                     n=5
    #                     if batch_idx % n == 0: # print every nth batch loss
    #                         print("\tbatch", batch_idx, "size", len(batch), "loss", loss.item())
            av_batch_loss = torch.Tensor(batch_loss).mean().item()
    #         print(f"ep{epoch} av_batch_loss\t {av_batch_loss}")
            loss_values.append(av_batch_loss)
            tepochs.set_postfix(av_batch_loss=av_batch_loss)
            if abs(av_batch_loss_running - av_batch_loss) < early_stopping_threshold:
                print("└────Early stopping.")
                break
            av_batch_loss_running = av_batch_loss
    print(" loss values:", *(f"{x:.3f}" for x in loss_values))
    if plot:
        plt.plot(loss_values)
        plt.show()
    print('╰────────Finished─training──\n')
    return loss_values

# train(rBorn, X, batchsize=30, max_epochs = 10, lr=0.1)

In [6]:
torch.autograd.set_detect_anomaly(True)

In [7]:
modelhom_loss_values={}
for model in models_hom:
    loss_values = train(model, X, batchsize=20, plot=False, max_epochs = 10, lr=0.1)
    plt.plot(loss_values, label=model.name)
    plt.ylabel('avg loss (NLL)')
    plt.xlabel('Epoch')
    plt.title(f"dataset: {dataset} (d={d}), bond dim={D}")
    modelhom_loss_values["model.name"]=loss_values
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

╭───────────────────────────
│Training Born (torch.float32), Homogeneous.


/Users/jonathanpalucci/anaconda3/envs/torch_nightly/lib/python3.8/site-packages/torch/autograd/__init__.py:144: UserWarning: Error detected in LogBackward. Traceback of forward call that caused the error:
  File "/Users/jonathanpalucci/anaconda3/envs/torch_nightly/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/jonathanpalucci/anaconda3/envs/torch_nightly/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/jonathanpalucci/anaconda3/envs/torch_nightly/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/jonathanpalucci/anaconda3/envs/torch_nightly/lib/python3.8/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/Users/jonathanpalucci/anaconda3/envs/torch_nightly/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 612, in start
    self.io_loop.start()
  File "

RuntimeError: Function 'LogBackward' returned nan values in its 0th output.

In [1]:
model_loss_values={}
for model in models:
    loss_values = train(model, X, batchsize=20, plot=False, max_epochs = 500, lr=0.1)
    plt.plot(loss_values, label=model.name)
    plt.ylabel('avg loss (NLL)')
    plt.xlabel('Epoch')
    plt.title(f"dataset: {dataset} (d={d}), bond dim={D}")
    model_loss_values["model.name"]=loss_values
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

NameError: name 'models' is not defined

---------

In [57]:
# testing complex functions
real = torch.tensor([[1, 2],[2, 1]], dtype=torch.float32)
imag = torch.tensor([[3, 4],[1, 2]], dtype=torch.float32)
z = torch.complex(real, imag)
# z = torch.randn(2,2, dtype=torch.cfloat)
z.requires_grad=True
b = (2.7j*z + 3.4*z + 18)
L = torch.einsum('ij,ji,ij,ji->',b,b,b,b).abs()
L.backward()

print("z:\n",z,'\n', sep='')
print("loss grad_fn:\n", L.grad_fn,'\n', sep='')
print("z.grad:\n", z.grad,'\n', sep='')

def infnorm(tensor):
#     return torch.max(torch.sum(abs(tensor), dim=-1))
    print(tensor.abs().max())
    return torch.linalg.norm(tensor, ord=float('inf')).real

norm = infnorm
print(norm(z.grad))
print(norm(z.grad[0]))

z:
tensor([[1.+3.j, 2.+4.j],
        [2.+1.j, 1.+2.j]], requires_grad=True)

loss grad_fn:

z.grad:
tensor([[103476.6328-38667.7930j, 224180.9375+59425.4219j],
        [219980.8125-67495.4922j, 105286.0781+37877.9062j]])

tensor(231923.4219)
tensor(342388.8438)
tensor(231923.4219)
tensor(231923.4219)


In [20]:
print(matnorm(z))
print(torch.max(torch.sum(abs(z), dim=1)))

tensor(7.6344+0.j, grad_fn=<CopyBackwards>)
tensor(7.6344, grad_fn=<MaxBackward1>)
